This tutorial can be downloaded [link](https://github.com/west-code-development/West/raw/master/Doc/tutorials/basic/basic_009.ipynb).

# Intro Tutorial 9: GW Calculations with Hybrid Functionals

This tutorial describes how to carry out $G_0W_0$ calculations with hybrid exchange-correlation functionals. The general steps are the same as those needed for $G_0W_0$ calculations with semi-local functionals, introduced in [Tutorial 1](https://github.com/west-code-development/West/raw/master/Doc/tutorials/basic/basic_001.ipynb).

## Exact exchange operator without approximation

We first perform the mean-field electronic structure calculation within DFT using the [Quantum ESPRESSO](https://www.quantum-espresso.org/) code. Download the following files to your working directory:

In [ ]:
%%bash
wget -N -q https://west-code.org/doc/training/exx/pw.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.2.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/Si_ONCV_PBE-1.2.upf

Let's inspect the `pw.in` file, input for `pw.x`.

In [1]:
%%bash
cat pw.in

&control
calculation  = 'scf'
restart_mode = 'from_scratch'
pseudo_dir   = './'
outdir       = './'
prefix       = 'silane'
wf_collect   = .TRUE.
/
&system
ibrav           = 1
celldm(1)       = 20
nat             = 5
ntyp            = 2
ecutwfc         = 25.0
nbnd            = 10
assume_isolated = 'mp'
input_dft       = 'pbe0'
/
&electrons
diago_full_acc = .TRUE.
/
ATOMIC_SPECIES
Si 28.0855  Si_ONCV_PBE-1.2.upf
H  1.00794   H_ONCV_PBE-1.2.upf
ATOMIC_POSITIONS bohr
Si      10.000000   10.000000  10.000000
H       11.614581   11.614581  11.614581
H        8.385418    8.385418  11.614581
H        8.385418   11.614581   8.385418
H       11.614581    8.385418   8.385418
K_POINTS {gamma}


Compared to the `pw.in` file used in [Tutorial 1](https://github.com/west-code-development/West/raw/master/Doc/tutorials/basic/basic_001.ipynb), here we only have one additonal keyword, `input_dft = 'pbe0'`, which instructs the code to use the PBE0 hybrid functional.

We run `pw.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out

The static dielectric screening is computed using the projective dielectric eigendecomposition (PDEP) technique. Download the following file to your working directory:

In [ ]:
%%bash
wget -N -q https://west-code.org/doc/training/exx/wstat.in

Let's inspect the `wstat.in` file.

In [2]:
%%bash
cat wstat.in

input_west:
    qe_prefix: silane
    west_prefix: silane
    outdir: ./

wstat_control:
    wstat_calculation: S
    n_pdep_eigen: 50


It's identical to the `wstat.in` file used in [Tutorial 1](https://github.com/west-code-development/West/raw/master/Doc/tutorials/basic/basic_001.ipynb).

We run `wstat.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 wstat.x -i wstat.in > wstat.out

The GW electronic structure is computed treating the frequency integration of the correlation part of the self energy with the contour deformation techinique and by computing the dielectric screening at multipole frequencies with Lanczos iterations. Download the following file to your working directory:

In [ ]:
%%bash
wget -N -q https://west-code.org/doc/training/exx/wfreq.in

Let's inspect the `wfreq.in` file.

In [3]:
%%bash
cat wfreq.in

input_west:
    qe_prefix: silane
    west_prefix: silane
    outdir: ./

wstat_control:
    wstat_calculation: S
    n_pdep_eigen: 50

wfreq_control:
    wfreq_calculation: XWGQ
    n_pdep_eigen_to_use: 50
    qp_bandrange: [1,5]
    n_refreq: 300
    ecut_refreq: 2.0


It's identical to the `wfreq.in` file used in [Tutorial 1](https://github.com/west-code-development/West/raw/master/Doc/tutorials/basic/basic_001.ipynb).

We run `wfreq.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 wfreq.x -i wfreq.in > wfreq.out

We save the output file `silane.wfreq.save/wfreq.json`.

In [ ]:
%%bash
mv silane.wfreq.save/wfreq.json wfreq_exx.json

If the reader does NOT have the computational resources to run the calculations, the output file can be directly downloaded as:

In [ ]:
%%bash
wget -N -q https://west-code.org/doc/training/exx/wfreq_exx.json

## Adaptively compressed exchange (ACE) operator

Now let's use the adaptively compressed exchange (ACE) operator to speed up the calculations. Download the following files to your working directory:

In [ ]:
%%bash
wget -N -q https://west-code.org/doc/training/exx/pw.in
wget -N -q https://west-code.org/doc/training/exx/nscf.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.2.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/Si_ONCV_PBE-1.2.upf

We run `pw.x` on 2 cores, and make a copy of the save directory generated by `pw.x`.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out
cp -r silane.save silane_copy.save

We run a non-self-consistent calculation to constuct the ACE operator with a bit more empty bands. The content of `nscf.in` is almost identical to that of `pw.in`, except that in `nscf.in` we request a non-self-consistent calculation with a larger value of `nbnd`.

We run `pw.x` again on 2 cores, and copy the file that stores the ACE operator to the directory saved in the previous self-consistent `pw.x` run.

In [ ]:
%%bash
mpirun -n 2 pw.x -i nscf.in > nscf.out
cp silane.save/ace* silane_copy.save
rm -rf silane.save
mv silane_copy.save silane.save

By doing this, in the `silane.save` directory, we have the wavefunctions generated in the first, self-consistent `pw.x` run and the ACE operator generated in the second, non-self-consistent `pw.x` run. Now we do the GW calculation. Download the following file to your working directory:

In [ ]:
%%bash
wget -N -q https://west-code.org/doc/training/exx/wstat_ace.in
wget -N -q https://west-code.org/doc/training/exx/wfreq_ace.in

Let's inspect the `wstat_ace.in` and `wfreq_ace.in` files.

In [4]:
%%bash
cat wstat_ace.in
cat wfreq_ace.in

input_west:
    qe_prefix: silane
    west_prefix: silane
    outdir: ./

wstat_control:
    wstat_calculation: S
    n_pdep_eigen: 50
    n_exx_lowrank: 30
input_west:
    qe_prefix: silane
    west_prefix: silane
    outdir: ./

wstat_control:
    wstat_calculation: S
    n_pdep_eigen: 50
    n_exx_lowrank: 30

wfreq_control:
    wfreq_calculation: XWGQ
    n_pdep_eigen_to_use: 50
    qp_bandrange: [1,5]
    n_refreq: 300
    ecut_refreq: 2.0


The new keyword, `n_exx_lowrank: 30`, activates the use of the ACE operator constructed from 30 KS wavefunctions. In production runs, `n_exx_lowrank` and therefore `nbnd` used on the non-self-consistent `pw.x` run should be converged.

We run `wstat.x` and `wfreq.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 wstat.x -i wstat_ace.in > wstat_ace.out
mpirun -n 2 wfreq.x -i wfreq_ace.in > wfreq_ace.out

We save the output file `silane.wfreq.save/wfreq.json`.

In [ ]:
%%bash
mv silane.wfreq.save/wfreq.json wfreq_ace.json

If the reader does NOT have the computational resources to run the calculations, the output file can be directly downloaded as:

In [ ]:
%%bash
wget -N -q https://west-code.org/doc/training/ace/wfreq_ace.json

We compare the quasiparticle corrections obtained with and without using the ACE operator.

In [5]:
import json
import numpy as np

def read_qp(filename):
    with open(filename) as file :
        data = json.load(file)

    return np.array(data['output']['Q']['K000001']['eqpSec'],dtype='f8')

qp_exx = read_qp('wfreq_exx.json')
qp_ace = read_qp('wfreq_ace.json')

print('QP energy [eV]')
print('State      EXX      ACE     Diff')
for i in range(5):
    print(f'    {i+1}  {qp_exx[i]:7.3f}  {qp_ace[i]:7.3f}  {qp_ace[i]-qp_exx[i]:7.3f}')

QP energy [eV]
State      EXX      ACE     Diff
    1  -17.363  -17.366   -0.003
    2  -12.365  -12.359    0.006
    3  -12.365  -12.359    0.006
    4  -12.367  -12.362    0.005
    5    0.383    0.380   -0.003


In this case, the difference between the EXX and ACE results is at most 6 meV.

In addition to $G_0W_0$ calculations, the ACE operator can also be used in Bethe-Salpeter equation (BSE), quantum defect embedding theory (QDET), and time-dependent density functional theory (TDDFT) calculations with hybrid functionals.